# MQTT Subscribe 測試應用程式

這個 notebook 示範如何使用 paho-mqtt 套件來訂閱 MQTT 訊息並接收訊息。


In [3]:
# 導入必要的套件
import paho.mqtt.client as mqtt
import time
import json


In [4]:
# MQTT Broker 設定
BROKER = "localhost"  # 或使用 "127.0.0.1"
PORT = 1883  # 標準 MQTT 埠，如果您的 Broker 使用不同埠號請修改
TOPIC = "客廳/溫度"  # 您可以修改這個主題名稱

# 建立 MQTT 客戶端（使用新的回調 API 版本 2）
client = mqtt.Client(callback_api_version=mqtt.CallbackAPIVersion.VERSION2)

# 連線回調函數（VERSION2 API）
def on_connect(client, userdata, flags, reason_code, properties):
    if reason_code.is_failure:
        print(f"❌ 連線失敗，錯誤代碼: {reason_code}")
    else:
        print(f"✅ 成功連接到 MQTT Broker: {BROKER}")
        # 連線成功後訂閱主題
        client.subscribe(TOPIC)
        print(f"已訂閱主題: {TOPIC}")

# 訊息接收回調函數（VERSION2 API）
def on_message(client, userdata, msg):
    print(f"📦 收到訊息 - 主題: {msg.topic}, 內容: {msg.payload.decode()}")
    try:
        # 嘗試解析 JSON 訊息
        json_data = json.loads(msg.payload.decode())
        print(f"   (JSON 解析): {json_data}")
    except json.JSONDecodeError:
        pass # 如果不是 JSON 格式，則不做處理

# 設定回調
client.on_connect = on_connect
client.on_message = on_message

# 連接到 Broker
print(f"正在連接到 {BROKER}...")
client.connect(BROKER, PORT, 60)

# 開始背景執行緒處理訊息
client.loop_start()  # 改為背景執行緒

# 等待連線建立
time.sleep(1)


正在連接到 localhost...
✅ 成功連接到 MQTT Broker: localhost
已訂閱主題: 客廳/溫度


In [5]:
# 設定訂閱時間（秒）
subscribe_duration = 10 # 訂閱 10 秒

print(f"開始監聽訊息，持續 {subscribe_duration} 秒...")
start_time = time.time()

while (time.time() - start_time) < subscribe_duration:
    # 在這裡可以執行其他任務，或者只是等待
    time.sleep(1)  # 每秒檢查一次

print(f"\n已停止監聽訊息 {subscribe_duration} 秒。")


開始監聽訊息，持續 10 秒...

已停止監聽訊息 10 秒。


## 監聽訊息

執行上方的 Cell，訂閱者將會監聽指定時間內的 MQTT 訊息。

您可以修改 `subscribe_duration` 變數來調整監聽時間。


In [6]:
# 關閉連線
client.loop_stop()
client.disconnect()
print("✅ MQTT 連線已關閉")


✅ MQTT 連線已關閉
